In [ ]:
import random
import numpy as np
import os, time

# Based on : 
# https://automaticaddison.com/value-iteration-vs-q-learning-algorithm-in-python-step-by-step/np%20ar

class SnakesAndLadders:

    DEBUG = False

    DICE_SAFE = {"index": 1, "name": "SAFE", "values": [0,1], "immune": 1.0}
    DICE_NORMAL = {"index": 2, "name": "NORMAL", "values": [0,1,2], "immune": 0.5}
    DICE_RISKY = {"index": 3, "name": "RISKY", "values": [0,1,2,3], "immune": 0.0}

    NOT_TRAP, TRAP_RESTART, TRAP_PENALTY, TRAP_PRISON, TRAP_GAMBLE = range(0, 5)

    # Indexes start at 0
    BOARD = {
        2: [3,10],
        9: 14
    }

    # ==============

    def __init__(self, layout: np.ndarray, circle: bool = False):
        self.layout = layout
        self.circle = circle
        self.length = len(layout)-1

        self.actions = [self.DICE_SAFE, self.DICE_NORMAL, self.DICE_RISKY]

    def isEnd(self, state: int): return state == self.length        

    def getStates(self):
        return range(len(self.layout))

    def getActions(self):
        return self.actions

    def setActions(self, actions: list):
        self.actions = actions

    # ==============

    # This corresponds to a multidimensional matrix of size (15, 3)
    # With values formatted as tuple : (nextState, probability, cost)
    def custom_transition_matrix(self):
        matrix = np.zeros((len(self.layout), len(self.actions)), dtype=object)

        for state in self.getStates():
            for action in self.getActions():
                matrix[state, action.get('index')-1] = self.nextProbCost(state, action)

        return matrix


    # Compute the cost, the probability and for all possible (future) states
    # for a given state and a given action
    def nextProbCost(self, state: int, action: int):

        returns = []

        immune = action.get('immune')
        values = action.get('values') # dice values
        probs = 1/len(values) # ex: [0,1,2] => prob is 1/3 of each value

        # For each possible value rolled by the dice
        for steps in values: 
            current = state

            # If we don't move, we only check for trap
            if steps == 0:
                returns += self.activateTrap(current, probs, immune)
            else: 
                # Otherwise, we do a first step. See forwardStep !
                # straight=False means that we can take an intersection
                possible = self.forwardStep(state, 1, False)
                for path in possible:
                    # For every path possible, we go forward (straight=True) by the number of steps left
                    nextState = self.forwardStep(path, steps-1, True)[0]
                    nextState = self.dontOverflow(nextState) # Handling the board end and "circle" parameter
                    returns += self.activateTrap(nextState, probs/len(possible), immune)
                    # the probability is now P(dice value & taking the path)
            
        if self.DEBUG: print("RETURNS for action", action.get("name"), ": ", returns, "\n")
        return returns

    def dontOverflow(self, state: int):
        if state > self.length: # do not overflow
            if self.circle: state -= self.length # if circle, get back to start
            else: state = self.length # otherwise, we're at the end
        return state

    def activateTrap(self, state: int, probs: float, immune: float):
        square = self.layout[state] # return the int value of the layout
        inactive, active = probs * immune, probs * (1-immune) # conditional probabilities

        if square == self.NOT_TRAP or immune == 1:
            return [(state, probs, 1)]

        elif square == self.TRAP_PRISON:
            return [(state, active, 2), (state, inactive, 1)] # cost is 2 if prison (bc 2 turns)

        elif square == self.TRAP_RESTART:
            return [(0, active, 1), (state, inactive, 1)]

        elif square == self.TRAP_GAMBLE:
            # We compute the probability of being teleported to any case
            # That's 1/nb_of_case multiplied by the prob of activation
            tp_probs = 1/len(self.layout)
            returns = [(nextState, active*tp_probs, 1) for nextState in range(len(self.layout))]
            return returns + [(state, inactive, 1)]

        elif square == self.TRAP_PENALTY:
            nextState = self.reverseStep(state, 3) # we step back by 3
            return [(nextState, active, 1), (state, inactive, 1)]

        else: 
            if self.DEBUG: print("UNEXPECTED: ", square)
            return [(state, 0, 0)]

    # ==============

    # Recursive
    def forwardStep(self, state, steps=1, straight=True):
        
        if steps <= 0: 
            if self.DEBUG: print("Forward: ", state, steps, straight)
            if type(state) is list: return state
            else: return [state]

        split = self.BOARD.get(state) # check if thats a special case
        if type(split) is int: nextState = split # if we jump
        elif type(split) is list and not straight: nextState = split # if thats a split
        else: nextState = state+1 # otherwise just step 1

        return self.forwardStep(nextState, steps-1, True)

    # Recursive
    def reverseStep(self, state, steps=1):
        if steps <= 0: 
            if self.DEBUG: print("Backward: ", state, steps)
            return state

        # Match the intersections (self.BOARD) but backward
        # basically we find key by value of a dict
        nextState = state - 1
        for k,v in self.BOARD.items():
            if v == state: nextState = k
            if type(v) == list: 
                if state in v: nextState =  k

        return self.reverseStep(max(nextState, 0), steps-1)

    # ==============


In [ ]:
def markovDecision(layout: np.ndarray, circle: bool = False):
    sal = SnakesAndLadders(layout, circle)
    
    # ==============
    V = np.ones(len(layout))
    V[sal.length] = 0 # end

    # ==============

    threshold = 1e-6
    done, failed, iteration, max_iterations = False, False, 0, 1000
    
    # ==============

    # Instead of computing it every time, we compute it once
    next_prob_cost_matrix = sal.custom_transition_matrix()

    def Q(state, action):
        # Formula : see slide 121 (p61) 
        index = action.get("index")-1

        # I tried something lol return prob_matrix[index, state] @ (cost_matrix[index, state] + V)

        return sum(prob * (cost + V[nextState]) \
                   for nextState, prob, cost in next_prob_cost_matrix[state, index])

    # ==============
    while not done: 
        iteration+=1

        v = np.copy(V)
        policy = {}

        # For all possible states
        for state in sal.getStates():

            if not sal.isEnd(state): 
                actions = sal.getActions()
                Qval = [Q(state, action) for action in actions]

                V[state] = min(Qval)
                policy[state] = actions[np.argmin(Qval)]

                if sal.DEBUG: print("It:", iteration, "||", V[state], policy[state].get("name"))
            else: 
                V[state] = 0
                policy[state] = {"name": "END"}    
    
        delta = max(abs(V[state] - v[state]) for state in sal.getStates())
        if delta <= threshold: done = True
        if iteration >= max_iterations: 
            done = True
            failed = True

        # ==============            

    # ==============
    # Printing (for debug purposes)
    if done: 
        print("\033[1m{:^15} {:^15} {:^15}\033[0m".format("Case", "V(s)", "Best"))
        for state in sal.getStates():
            print("{:^15} {:^15.3} {:^15}".format(state+1, V[state], policy[state].get("name")))
        print("")
        

    if failed : print("\033[91m", "[WARNING] The algorithm was unable to converge in", max_iterations, "iterations and a threshold of", threshold, "..", "\033[0m")
    else: print("\033[92m", "[INFO] Convergence achieved with a threshold of", threshold, "in", iteration, "iterations.", "\033[0m")

    policy = np.array([p.get("index") for p in policy.values()])
    return [V[:sal.length], policy[:sal.length]]


In [ ]:

# Building the layout with SnakesAndLadders
# only because its easier but thats just integer in reality 
layout = np.array([
    SnakesAndLadders.NOT_TRAP, # 1 START
    SnakesAndLadders.NOT_TRAP, # 2
    SnakesAndLadders.NOT_TRAP, # 3 SPLIT

    SnakesAndLadders.NOT_TRAP, # 4: SPLIT-1
    SnakesAndLadders.NOT_TRAP, # 5
    SnakesAndLadders.NOT_TRAP, # 6
    SnakesAndLadders.NOT_TRAP, # 7
    SnakesAndLadders.NOT_TRAP, # 8
    SnakesAndLadders.NOT_TRAP, # 9
    SnakesAndLadders.TRAP_GAMBLE, # 10

    SnakesAndLadders.NOT_TRAP, # 11 : SPLIT 2
    SnakesAndLadders.NOT_TRAP, # 12
    SnakesAndLadders.NOT_TRAP, # 13
    SnakesAndLadders.TRAP_GAMBLE, # 14
    
    SnakesAndLadders.NOT_TRAP, # 15 END
])


# Just for style
#os.system("cls")
optimal = markovDecision(layout, False)
print("\033[92m", "[INFO] The layout provided was:", layout, "\033[0m")
print("")
print("\033[93m", "[INFO] Optimal solution found:", optimal[1],  "\033[0m")
print("\033[93m", "[INFO] Average number of turns:", optimal[0],  "\033[0m")

In [ ]:
# TODO
class Simulation(SnakesAndLadders):

    def __init__(layout, circle, strategy): #strategy : vector of length 15, each element is the dice to use on the square
        self.layout = layout
        self.circle = circle
        self.strategy = strategy

    def play(layout, circle, strategy): #returns number of turns to complete game
        current = 0 #current square
        nturns = 0  #number of turns 
        while(current != 14) :
          current = goNextSquare(current, layout, circle, strategy)
          current = activateTrap(current)
        return nturns  

    def goNextSquare(current, nturns, layout, circle, strategy) :
      dice = strategy[current]  
      if(current != 3) :
        if(dice = SnakesAndLadders.DICE_SAFE) :
            dice_value = random.randint(0, 1)
            current = current + dice_value
            

    
    def expectedTurns(layout, circle, strategy) : #returns mean number of turns to complete the game
      turns_vector = []
      for i in range(1000) :
        turns_vector.append(self.play(layout, cirlce, strategy))
      return np.mean(turns_vector) 

        

    

In [ ]:
# TEST ZONE
# =========
sal = SnakesAndLadders(layout, False)
sal.DEBUG = True

sim = Simulation
print(sim.generateLayout(15, [sal.TRAP_GAMBLE, sal.TRAP_PENALTY, sal.TRAP_PRISON, sal.TRAP_RESTART], 5))

TypeError: ignored